In [44]:
import os
import re
from pathlib import Path
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

In [49]:
# Parameter
input_dir = Path("data/processed")
chunk_size = 200
chunk_overlap = 0
index_path = "faiss_index"
model_name = "sentence-transformers/all-MiniLM-L6-v2"

In [50]:
# Text splitter konfigurieren
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    separators=["\n\n", "\n", ".", " ", ""]
)

In [51]:
# Alle Dokumente sammeln
all_documents = []

for md_file in input_dir.glob("*.md"):

    # Dateiname analysieren
    filename = md_file.stem  # z. B. "bmw_2023_filtered"
    parts = filename.split("_")  # → ["bmw", "2023", "filtered"]

    # Metadaten extrahieren
    company = parts[0]
    year = parts[1]

    with md_file.open("r", encoding="utf-8") as f:
        text = f.read()

    text = text.replace('\u202f', ' ') # Filter 1: Unicode entfernen
    text = re.sub(r'(?<=\d)\.(?=\d)', '', text) # Filter 2: Entferne Punkte zwischen zwei Ziffern (z. B. 1.234 → 1234)

    chunks = text_splitter.split_text(text)
    
    for i, chunk in enumerate(chunks):
        all_documents.append(Document(
            page_content=chunk,
            metadata={"company": company, "year": year, "chunk": i}
        ))

In [52]:
bmw_chunks = [doc for doc in all_documents if doc.metadata["company"].lower() == "bmw"]
bmw_chunks[-20:]

[Document(metadata={'company': 'BMW', 'year': '2023', 'chunk': 6912}, page_content='. In den Bereichen Digitalisierung, Nachhaltigkeit und Kundenzufriedenheit lagen die Zielerreichungen zwischen 70 und 80 Prozent.'),
 Document(metadata={'company': 'BMW', 'year': '2023', 'chunk': 6913}, page_content='Die Kennzahlenentwicklung der letzten fünf Jahre zeigt eine positive Gesamttendenz mit teils starken Schwankungen'),
 Document(metadata={'company': 'BMW', 'year': '2023', 'chunk': 6914}, page_content='. Die Fahrzeugauslieferungen stiegen von 2185793 Einheiten im Jahr 2019 auf 2554183 im Jahr 2023'),
 Document(metadata={'company': 'BMW', 'year': '2023', 'chunk': 6915}, page_content='. Das Konzernergebnis vor Steuern wuchs von 7118 Millionen Euro auf einen Höchststand von 23509 Millionen Euro im Jahr 2022 und lag 2023 bei 17096 Millionen Euro'),
 Document(metadata={'company': 'BMW', 'year': '2023', 'chunk': 6916}, page_content='. Der Free Cashflow im Segment Automobile schwankte stark: von 25

In [53]:
# Embeddings vorbereiten
embeddings = HuggingFaceEmbeddings(model_name=model_name)

In [54]:
# FAISS-Index erstellen
faiss_index = FAISS.from_documents(all_documents, embedding=embeddings)

In [55]:
# Lokal speichern
faiss_index.save_local(index_path)